In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as clr
import matplotlib.dates as md
import seaborn as sns
import datetime as datetime
import numpy as np
import os

%matplotlib inline

plt.style.use('default')

sns.set(style="whitegrid")

import warnings 
warnings.filterwarnings('ignore')

my_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [17]:
#CORRER PRIMERO EL NOTEBOOK FEATURES PARA CREAR LOS CSV DE LABELS Y FEATURES

labels = []
features_por_ventana = []

for i in range(0,5):
    labels.append(pd.read_csv('labels_v'+str(i+1)+'.csv'))
    labels[i].drop('Unnamed: 0',axis='columns',inplace=True)
    features_por_ventana.append(pd.read_csv('features_v'+str(i+1)+'.csv'))
    features_por_ventana[i].drop('Unnamed: 0',axis='columns',inplace=True)

In [18]:
labels[0].head()

,device_id,st,sc
0,1826643666390887030,0.015050,259200.000
1,7037174172278258682,0.029014,259200.000
2,3392065368947589877,0.057540,259200.000
3,1228982273563226229,0.126828,259200.000
4,4123059034628125459,0.132510,17800.664


In [19]:
features_por_ventana[0].head()

,device_id,index_x,cantidad_de_subastas_en_ventana,cantidad_subastas_Monday,cantidad_subastas_Tuesday,cantidad_subastas_Wednesday,cantidad_subastas_Thursday,cantidad_subastas_Friday,cantidad_subastas_Saturday,cantidad_subastas_Sunday,...,cantidad_eventos_hora_15,cantidad_eventos_hora_16,cantidad_eventos_hora_17,cantidad_eventos_hora_18,cantidad_eventos_hora_19,cantidad_eventos_hora_20,cantidad_eventos_hora_21,cantidad_eventos_hora_22,cantidad_eventos_hora_23,device_os_version
0,41863526108385,0.0,35.0,0.0,0.0,0.0,0.0,1.0,34.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0
1,135153013040192,1.0,8.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,161514654074162,2.0,6.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,...,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,181891380775191,3.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,186034136943920,4.0,7.0,0.0,0.0,0.0,2.0,0.0,5.0,0.0,...,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Label

In [20]:
labels_b = []
tiempo_maximo = 259200

for i in range(0,5):
    labels[i] = (labels[i].loc[labels[i]['device_id'].isin(features_por_ventana[i]['device_id'])])[['device_id','st']]
    labels[i]['binary'] = np.where(labels[i]['st']>=tiempo_maximo,False,True)
    labels[i].sort_values(by='device_id',inplace=True)
    labels_b.append(labels[i][['binary','st']].to_records(index=False))

In [21]:
labels_b[0]

rec.array([( True, 157228.465866), ( True, 187854.009137),
           ( True,  10366.357746), ..., ( True,  52347.139395),
           ( True,  17194.358485), ( True,  50088.250389)],
          dtype=[('binary', '?'), ('st', '<f8')])

# Se agrega chequeo para que los df_feautures cumplan condiciones en los algoritmos a ser usados

In [22]:
for i in range(0,5):
    if (features_por_ventana[i].shape[0] != labels[i].shape[0]):
        raise AssertionError("Los df_features de cada ventana tienen que tener misma \
                cantidad de dispositivos(mismos device_id)")

# Algoritmo

# GradientBoostingSurvivalAnalysis

In [ ]:
from sksurv.column import categorical_to_numeric
for i in range(0,5):
    features_por_ventana[i].replace([np.inf, -np.inf],0,inplace=True)
    features_por_ventana[i]['tipo_evento'] = features_por_ventana[i]['tipo_evento'].astype('category')
    features_por_ventana[i]['wifi'] = features_por_ventana[i]['wifi'].astype('category')
    features_por_ventana[i]['kind'] = features_por_ventana[i]['kind'].astype('category')
    features_por_ventana[i]['device_os_version'] = features_por_ventana[i]['device_os_version'].astype('category')
    features_por_ventana[i]['prediccion_ventana_0'] = 0
    features_por_ventana[i]['prediccion_ventana_1'] = 0
    features_por_ventana[i]['prediccion_ventana_2'] = 0
    features_por_ventana[i]['prediccion_ventana_3'] = 0
    features_por_ventana[i]['prediccion_ventana_4'] = 0
    features_por_ventana[i]['prediccion_ventana_5'] = 0
    features_por_ventana[i] = categorical_to_numeric(features_por_ventana[i])
    features_por_ventana[i].fillna(0,inplace=True)
    
    

In [ ]:
from sksurv.ensemble import GradientBoostingSurvivalAnalysis

score = []
for i in range(0,4):
    model = GradientBoostingSurvivalAnalysis()
    model.fit(features_por_ventana[i],labels_b[i])
    #prediccion = model.predict(features_por_ventana[i+1])
    #features_por_ventana[i+1]['prediccion_ventana_'+str(i)] = pd.Series(prediccion)
    score.append( model.score(features_por_ventana[i+1], labels_b[i+1]) )
    
    
    

In [15]:
from sksurv.preprocessing import OneHotEncoder

features_v1 = OneHotEncoder().fit_transform(features_por_ventana[0])
features_v2 = OneHotEncoder().fit_transform(features_por_ventana[1])
features_v1.head()

,device_id,cantidad_de_subastas_en_ventana,cantidad_subastas_Monday,cantidad_subastas_Tuesday,cantidad_subastas_Wednesday,cantidad_subastas_Thursday,cantidad_subastas_Friday,cantidad_subastas_Saturday,cantidad_subastas_Sunday,cantidad_de_clicks_en_ventana,...,cantidad_eventos_hora_15,cantidad_eventos_hora_16,cantidad_eventos_hora_17,cantidad_eventos_hora_18,cantidad_eventos_hora_19,cantidad_eventos_hora_20,cantidad_eventos_hora_21,cantidad_eventos_hora_22,cantidad_eventos_hora_23,device_os_version
0,41863526108385,35.0,0.0,0.0,0.0,0.0,1.0,34.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0
1,135153013040192,8.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,161514654074162,6.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,...,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,181891380775191,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,186034136943920,7.0,0.0,0.0,0.0,2.0,0.0,5.0,0.0,0.0,...,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [64]:
from sksurv.ensemble import GradientBoostingSurvivalAnalysis

model = GradientBoostingSurvivalAnalysis()

In [ ]:
model.fit(features_v1.head(5000),labels_b[0][:5000])

In [60]:
model.score(features_v2.head(5000),labels_b[1][:5000])

0.8629979777886098

In [12]:
from sksurv.preprocessing import OneHotEncoder
for i in range(0,5):
    features_por_ventana[i] = OneHotEncoder().fit_transform(features_por_ventana[i])

In [ ]:
from sksurv.ensemble import GradientBoostingSurvivalAnalysis

score = []
for i in range(0,1):
    model = GradientBoostingSurvivalAnalysis()
    model.fit(features_por_ventana[i].head(5000),labels_b[i][:5000])
    prediccion = model.predict(features_por_ventana[i+1].head(5000))
    features_por_ventana[i+1]['prediccion_ventana_'+str(i)] = prediccion
    score.append( model.score(features_por_ventana[i+1].head(5000), labels_b[i+1][:5000]) )
    
    
    

In [16]:
from sksurv.linear_model import CoxPHSurvivalAnalysis

estimator = CoxPHSurvivalAnalysis(alpha=0.0001)
estimator.fit(features_v1.head(10000),labels_b[0][:10000])

CoxPHSurvivalAnalysis(alpha=0.0001, n_iter=100, tol=1e-09, verbose=0)

In [17]:
pd.Series(estimator.coef_, index=features_v1.columns)

device_id                              -1.519372e-49
cantidad_de_subastas_en_ventana         1.427998e-32
cantidad_subastas_Monday                0.000000e+00
cantidad_subastas_Tuesday               0.000000e+00
cantidad_subastas_Wednesday             0.000000e+00
cantidad_subastas_Thursday              3.456665e-32
cantidad_subastas_Friday               -5.852128e-33
cantidad_subastas_Saturday             -1.452607e-32
cantidad_subastas_Sunday                0.000000e+00
cantidad_de_clicks_en_ventana           1.088082e-31
cantidad_clicks_Monday                  0.000000e+00
cantidad_clicks_Tuesday                 0.000000e+00
cantidad_clicks_Wednesday               0.000000e+00
cantidad_clicks_Thursday                5.264689e-31
cantidad_clicks_Friday                 -7.815852e-32
cantidad_clicks_Saturday               -3.395026e-31
cantidad_clicks_Sunday                  0.000000e+00
tiempo_de_click_promedio                2.896356e-36
SUP_D                                   4.8901

#### El resultado es un vector de coeficientes, uno para cada variable, donde cada valor corresponde a la relación del registro y evento

In [18]:
estimator.score(features_v2.head(10000), labels_b[1][:10000])

0.5

# KNN

In [19]:
from sklearn.metrics import mean_squared_error

def root_mean_squared_error(y_test, predicted):
    return (np.sqrt(mean_squared_error(y_test,predicted)))

In [20]:
df = labels[0].sort_values(ascending=True,by='device_id').head(10000)
df.head()

,device_id,st,binary
257007,41863526108385,157228.465866,True
285766,135153013040192,187854.009137,True
70944,161514654074162,10366.357746,True
316713,181891380775191,256765.420614,True
155213,186034136943920,60166.331894,True


In [21]:
df_feat = features_v1.sort_values(ascending=True,by='device_id').head(10000)
df_feat.head()

,device_id,cantidad_de_subastas_en_ventana,cantidad_subastas_Monday,cantidad_subastas_Tuesday,cantidad_subastas_Wednesday,cantidad_subastas_Thursday,cantidad_subastas_Friday,cantidad_subastas_Saturday,cantidad_subastas_Sunday,cantidad_de_clicks_en_ventana,...,cantidad_eventos_hora_15,cantidad_eventos_hora_16,cantidad_eventos_hora_17,cantidad_eventos_hora_18,cantidad_eventos_hora_19,cantidad_eventos_hora_20,cantidad_eventos_hora_21,cantidad_eventos_hora_22,cantidad_eventos_hora_23,device_os_version
0,41863526108385,35.0,0.0,0.0,0.0,0.0,1.0,34.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0
1,135153013040192,8.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,161514654074162,6.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,...,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,181891380775191,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,186034136943920,7.0,0.0,0.0,0.0,2.0,0.0,5.0,0.0,0.0,...,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
from sklearn.model_selection import train_test_split

#TRAIN_TEST_SPLIT
#Split arrays or matrices into random train and test subsets
x_train, x_test, y_train, y_test = train_test_split(df_feat.drop(columns='device_id'),
                                                    df['st'],
                                                    test_size=0.2,
                                                    random_state=43)

In [23]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn import metrics

k=5 #-->Hiperparametro
neigh = KNeighborsRegressor(n_neighbors = k ,    #Cantidad de vecinos a usar --> K hiperparametro.
                            n_jobs=-1,           #Define la cantidad de procesadores a usar, -1 todos los disponibles.
                            weights='distance',  #Se define el peso como la inversa a la distancia, los mas cercanos tienen mayor influencia.
                            algorithm='brute'    #Fuerza bruta para el algorimo, podría ser KDTree o sino automatico.
                           )
#En weights se puede pasar una funcion definida por el usuario--> Segundo hiperparametro posible de KNN.

In [24]:
neigh.fit(x_train,y_train) #Fit the model using X as training data and Y as target values

KNeighborsRegressor(algorithm='brute', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
          weights='distance')

In [25]:
predicted_knn = neigh.predict(x_test) #Predict the target for the provided data

In [26]:
root_mean_squared_error(y_test,predicted_knn)

73721.73922380546

In [27]:
neigh.score(x_train, y_train)

0.5682946498767272

In [28]:
neigh.score(x_test, y_test)

0.4087005232845691

# Gradient Boosting

In [29]:
from sklearn.ensemble import GradientBoostingRegressor

gradient_boost = GradientBoostingRegressor()

In [30]:
gradient_boost.fit(x_train,y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, n_iter_no_change=None, presort='auto',
             random_state=None, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)

In [31]:
predicted_gb = gradient_boost.predict(x_test)

In [32]:
gradient_boost.score(x_train, y_train)

0.9372422830604865

In [33]:
gradient_boost.score(x_test, y_test)

0.935215001116724

In [34]:
root_mean_squared_error(y_test,predicted_gb)

24402.20396881875

In [63]:
from sklearn.model_selection import GridSearchCV
p_test1 = {'learning_rate':[0.15,0.1,0.05,0.01,0.005,0.001], 'n_estimators':[100,250,500,750,1000,1250,1500,1750]}

tuning = GridSearchCV(estimator =GradientBoostingRegressor(), param_grid = p_test1)

tuning.fit(df_feat.drop(columns='device_id'),df['st'])
tuning.best_params_, tuning.best_score_

({'learning_rate': 0.005, 'n_estimators': 1250}, 0.9335394614324747)

In [65]:
gradient_boost_2 = GradientBoostingRegressor(n_estimators= 1250, learning_rate=0.005)
gradient_boost_2.fit(x_train,y_train)
predicted_gb_2 = gradient_boost_2.predict(x_test)
gradient_boost_2.score(x_train, y_train)
root_mean_squared_error(y_test,predicted_gb_2)

24429.581248523762

In [67]:
gradient_boost_2.score(x_test, y_test)
#Baja

0.9350695526174552

In [71]:
p_test2 = {'max_depth':[2,3,4,5,6,7]}
tuning = GridSearchCV(estimator =GradientBoostingRegressor(), param_grid = p_test2)

tuning.fit(df_feat.drop(columns='device_id'),df['st'])
tuning.best_params_, tuning.best_score_

({'max_depth': 3}, 0.9338475449027231)

In [72]:
gradient_boost_3 = GradientBoostingRegressor(max_depth= 3)
gradient_boost_3.fit(x_train,y_train)
gradient_boost_3.score(x_test, y_test)
#baja

0.9351380720519836

In [73]:
p_test3 = {'min_samples_split':[2,4,6,8,10,20,40,60,100], 'min_samples_leaf':[1,3,5,7,9] }

tuning = GridSearchCV(estimator =GradientBoostingRegressor(), param_grid = p_test3)

tuning.fit(df_feat.drop(columns='device_id'),df['st'])
tuning.best_params_, tuning.best_score_

({'min_samples_leaf': 5, 'min_samples_split': 60}, 0.9339450691357554)

In [74]:
gradient_boost_4 = GradientBoostingRegressor(min_samples_leaf= 5, min_samples_split= 60)
gradient_boost_4.fit(x_train,y_train)
gradient_boost_4.score(x_test, y_test)
#Baja

0.9350079834773051

# Random Forest

In [35]:
from sklearn.ensemble import RandomForestRegressor

random_forest = RandomForestRegressor()

In [36]:
random_forest.fit(x_train,y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [37]:
predicted_rf = random_forest.predict(x_test)

In [38]:
root_mean_squared_error(y_test,predicted_rf)

26257.269977207576

In [39]:
random_forest.score(x_train, y_train)

0.9635473701858035

In [40]:
random_forest.score(x_test,y_test)

0.9249906349830139

# Linear Regression

In [41]:
from sklearn.linear_model import LinearRegression

reg_lineal = LinearRegression()

In [42]:
reg_lineal.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [43]:
reg_lineal.score(x_train,y_train)

0.071462249430705

In [44]:
reg_lineal.score(x_test,y_test)

0.07307400818712761

# Ada Boost

In [45]:
from sklearn.ensemble import AdaBoostRegressor

ada_boost = AdaBoostRegressor()

In [46]:
ada_boost.fit(x_train,y_train)

AdaBoostRegressor(base_estimator=None, learning_rate=1.0, loss='linear',
         n_estimators=50, random_state=None)

In [47]:
predicted_ab = ada_boost.predict(x_test)

In [48]:
root_mean_squared_error(y_test,predicted_ab)

24793.306589140786

In [49]:
ada_boost.score(x_train, y_train)

0.9324363488726378

In [50]:
ada_boost.score(x_test,y_test)

0.9331216959394626

# Decision Tree

In [51]:
from sklearn.tree import DecisionTreeRegressor

arbol_dec = DecisionTreeRegressor()

In [52]:
arbol_dec.fit(x_train,y_train)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [53]:
predicted_dt = arbol_dec.predict(x_test) 

In [54]:
root_mean_squared_error(y_test,predicted_dt)

31685.815430226838

In [55]:
arbol_dec.score(x_train,y_train)

0.9708939774401235

In [56]:
arbol_dec.score(x_test,y_test)

0.8907689455636174

In [57]:
feature_importances = pd.DataFrame(arbol_dec.feature_importances_,
                                   index = x_train.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)
feature_importances.head(10)

,importance
cantidad_subastas_Thursday,0.778212
cantidad_subastas_Friday,0.173654
cantidad_subastas_Saturday,0.023967
cantidad_de_subastas_en_ventana,0.008015
cantidad_de_eventos_en_ventana,0.001236
aplicacion_mas_usada,0.001119
tipo_evento_mas_frec,0.001060
Porc_I-E,0.000908
cantidad_instalaciones_Thursday,0.000735
cantidad_eventos_Friday,0.000727


# XGBoost

In [58]:
from xgboost import XGBRegressor

xgboost = XGBRegressor()

In [59]:
xgboost.fit(x_train,y_train)

[22:12:27] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity=1)

In [60]:
predicted_xg = xgboost.predict(x_test)

In [61]:
root_mean_squared_error(y_test,predicted_xg)

24445.925918910216

In [62]:
xgboost.score(x_train,y_train)

0.9370968849813937

In [63]:
xgboost.score(x_test,y_test)

0.9349826398070178

In [93]:
parameters_xgb = {
        'num_boost_round': [100, 250, 500],
        'eta': [0.05, 0.1, 0.3],
        'max_depth': [6, 9, 12],
        'subsample': [0.9, 1.0],
        'colsample_bytree': [0.9, 1.0],
}

tuning_xgb = GridSearchCV(estimator = XGBRegressor(), param_grid = parameters_xgb)

tuning_xgb.fit(df_feat.drop(columns='device_id'),df['st'])
tuning_xgb.best_params_, tuning.best_score_

[14:44:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:46] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:49] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:51] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:54] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:56] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:59] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:45:01] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:45:03] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

({'colsample_bytree': 0.9,
  'eta': 0.05,
  'max_depth': 6,
  'num_boost_round': 100,
  'subsample': 1.0},
 0.9339450691357554)

In [ ]:
'colsample_bytree': 1.0,
  'eta': 0.05,
  'max_depth': 6,
  'num_boost_round': 100,
  'subsample': 1.0

# Majority Voting

In [63]:
from sklearn.ensemble import VotingRegressor
from sklearn import model_selection

In [112]:
#Jugando con cuales se usan se obtienen distintos resultados
estimators = []
#estimators.append(('RandomForest',RandomForestRegressor()))
#estimators.append(('DecisionTree', DecisionTreeRegressor()))
estimators.append(('AdaBoost', AdaBoostRegressor()))
#estimators.append(('LinearRegression',LinearRegression()))
estimators.append(('XGBoost',XGBRegressor()))
#estimators.append(('GradientBoosting',GradientBoostingRegressor()))

In [113]:
voting = VotingRegressor(estimators)
voting.fit(x_train,y_train)
#results = model_selection.cross_val_score(voting,x_train,y_train)
#results
voting.score(x_test,y_test)

[19:38:43] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


0.9344179578326095